# Прекод

# Сборный проект-4

Вам поручено разработать демонстрационную версию поиска изображений по запросу.

Для демонстрационной версии нужно обучить модель, которая получит векторное представление изображения, векторное представление текста, а на выходе выдаст число от 0 до 1 — покажет, насколько текст и картинка подходят друг другу.

### Описание данных

Данные лежат в папке `/datasets/image_search/` или доступны по [ссылке](https://code.s3.yandex.net/datasets/dsplus_integrated_project_4.zip).

В файле `train_dataset.csv` находится информация, необходимая для обучения: имя файла изображения, идентификатор описания и текст описания. Для одной картинки может быть доступно до 5 описаний. Идентификатор описания имеет формат `<имя файла изображения>#<порядковый номер описания>`.

В папке `train_images` содержатся изображения для тренировки модели.

В файле `CrowdAnnotations.tsv` — данные по соответствию изображения и описания, полученные с помощью краудсорсинга. Номера колонок и соответствующий тип данных:

1. Имя файла изображения.
2. Идентификатор описания.
3. Доля людей, подтвердивших, что описание соответствует изображению.
4. Количество человек, подтвердивших, что описание соответствует изображению.
5. Количество человек, подтвердивших, что описание не соответствует изображению.

В файле `ExpertAnnotations.tsv` содержатся данные по соответствию изображения и описания, полученные в результате опроса экспертов. Номера колонок и соответствующий тип данных:

1. Имя файла изображения.
2. Идентификатор описания.

3, 4, 5 — оценки трёх экспертов.

Эксперты ставят оценки по шкале от 1 до 4, где 1 — изображение и запрос совершенно не соответствуют друг другу, 2 — запрос содержит элементы описания изображения, но в целом запрос тексту не соответствует, 3 — запрос и текст соответствуют с точностью до некоторых деталей, 4 — запрос и текст соответствуют полностью.

В файле `test_queries.csv` находится информация, необходимая для тестирования: идентификатор запроса, текст запроса и релевантное изображение. Для одной картинки может быть доступно до 5 описаний. Идентификатор описания имеет формат `<имя файла изображения>#<порядковый номер описания>`.

В папке `test_images` содержатся изображения для тестирования модели.

In [1]:
#!pip install spacy
#!python -m spacy download en_core_web_sm
#!pip install spacy[transformers]
#!pip install torchvision

In [72]:
import torchvision.models as models
import torch
import torch.nn as nn
import numpy as np
from tensorflow.python.training import py_checkpoint_reader
import tensorflow as tf

import pandas as pd
import random as rd

from PIL import Image
import os

import spacy
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import GroupShuffleSplit

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from skorch.callbacks import EpochScoring, EarlyStopping
from skorch import NeuralNetRegressor
from sklearn.pipeline import Pipeline

import torch
import torch.nn as nn
from math import ceil
from torchvision import transforms
from tqdm import notebook
from torch.utils.data import Dataset

## 1. Исследовательский анализ данных

Наш датасет содержит экспертные и краудсорсинговые оценки соответствия текста и изображения.

В файле с экспертными мнениями для каждой пары изображение-текст имеются оценки от трёх специалистов. Для решения задачи вы должны эти оценки агрегировать — превратить в одну. Существует несколько способов агрегации оценок, самый простой — голосование большинства: за какую оценку проголосовала большая часть экспертов (в нашем случае 2 или 3), та оценка и ставится как итоговая. Поскольку число экспертов меньше числа классов, может случиться, что каждый эксперт поставит разные оценки, например: 1, 4, 2. В таком случае данную пару изображение-текст можно исключить из датасета.

Вы можете воспользоваться другим методом агрегации оценок или придумать свой.

В файле с краудсорсинговыми оценками информация расположена в таком порядке:

1. Доля исполнителей, подтвердивших, что текст **соответствует** картинке.
2. Количество исполнителей, подтвердивших, что текст **соответствует** картинке.
3. Количество исполнителей, подтвердивших, что текст **не соответствует** картинке.

После анализа экспертных и краудсорсинговых оценок выберите либо одну из них, либо объедините их в одну по какому-то критерию: например, оценка эксперта принимается с коэффициентом 0.6, а крауда — с коэффициентом 0.4.

Ваша модель должна возвращать на выходе вероятность соответствия изображения тексту, поэтому целевая переменная должна иметь значения от 0 до 1.


In [3]:
train_dataset = pd.read_csv('to_upload/train_dataset.csv')
CrowdAnnotations = pd.read_csv('to_upload/CrowdAnnotations.tsv', sep='\t', header=None)
ExpertAnnotations = pd.read_csv('to_upload/ExpertAnnotations.tsv', sep='\t', header=None)
test_queries = pd.read_csv('to_upload/test_queries.csv', sep='|', index_col=0)
test_images = pd.read_csv('to_upload/test_images.csv', sep='|')

In [4]:
def display_table_info(name, data):
    print(name)
    display(data.head(10))
    data.info()
    print('')
    print('')
    print('')

display_table_info('train_dataset', train_dataset)
display_table_info('CrowdAnnotations', CrowdAnnotations)
display_table_info('ExpertAnnotations', ExpertAnnotations)
display_table_info('test_queries', test_queries)
display_table_info('test_images', test_images)

train_dataset


,image,query_id,query_text
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
1,1262583859_653f1469a9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
2,2447284966_d6bbdb4b6e.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
3,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
4,2621415349_ef1a7e73be.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
5,3030566410_393c36a6c5.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
6,3155451946_c0862c70cb.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
7,3222041930_f642f49d28.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
8,343218198_1ca90e0734.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
9,3718964174_cb2dc1615e.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5822 entries, 0 to 5821
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image       5822 non-null   object
 1   query_id    5822 non-null   object
 2   query_text  5822 non-null   object
dtypes: object(3)
memory usage: 136.6+ KB



CrowdAnnotations


,0,1,2,3,4
0,1056338697_4f7d7ce270.jpg,1056338697_4f7d7ce270.jpg#2,1.000000,3,0
1,1056338697_4f7d7ce270.jpg,114051287_dd85625a04.jpg#2,0.000000,0,3
2,1056338697_4f7d7ce270.jpg,1427391496_ea512cbe7f.jpg#2,0.000000,0,3
3,1056338697_4f7d7ce270.jpg,2073964624_52da3a0fc4.jpg#2,0.000000,0,3
4,1056338697_4f7d7ce270.jpg,2083434441_a93bc6306b.jpg#2,0.000000,0,3
5,1056338697_4f7d7ce270.jpg,2204550058_2707d92338.jpg#2,0.000000,0,3
6,1056338697_4f7d7ce270.jpg,2224450291_4c133fabe8.jpg#2,0.000000,0,3
7,1056338697_4f7d7ce270.jpg,2248487950_c62d0c81a9.jpg#2,0.333333,1,2
8,1056338697_4f7d7ce270.jpg,2307118114_c258e3a47e.jpg#2,0.000000,0,3
9,1056338697_4f7d7ce270.jpg,2309860995_c2e2a0feeb.jpg#2,0.000000,0,3


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47830 entries, 0 to 47829
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       47830 non-null  object 
 1   1       47830 non-null  object 
 2   2       47830 non-null  float64
 3   3       47830 non-null  int64  
 4   4       47830 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.8+ MB



ExpertAnnotations


...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5822 entries, 0 to 5821
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5822 non-null   object
 1   1       5822 non-null   object
 2   2       5822 non-null   int64 
 3   3       5822 non-null   int64 
 4   4       5822 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 227.6+ KB



test_queries


,query_id,query_text,image
0,1177994172_10d143cb8d.jpg#0,"Two blonde boys , one in a camouflage shirt an...",1177994172_10d143cb8d.jpg
1,1177994172_10d143cb8d.jpg#1,Two boys are squirting water guns at each other .,1177994172_10d143cb8d.jpg
2,1177994172_10d143cb8d.jpg#2,Two boys spraying each other with water,1177994172_10d143cb8d.jpg
3,1177994172_10d143cb8d.jpg#3,Two children wearing jeans squirt water at eac...,1177994172_10d143cb8d.jpg
4,1177994172_10d143cb8d.jpg#4,Two young boys are squirting water at each oth...,1177994172_10d143cb8d.jpg
5,1232148178_4f45cc3284.jpg#0,A baby girl playing at a park .,1232148178_4f45cc3284.jpg
6,1232148178_4f45cc3284.jpg#1,A closeup of a child on a playground with adul...,1232148178_4f45cc3284.jpg
7,1232148178_4f45cc3284.jpg#2,A young boy poses for a picture in front of a ...,1232148178_4f45cc3284.jpg
8,1232148178_4f45cc3284.jpg#3,A young girl is smiling in front of the camera...,1232148178_4f45cc3284.jpg
9,1232148178_4f45cc3284.jpg#4,There is a little blond hair girl with a green...,1232148178_4f45cc3284.jpg


<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   query_id    500 non-null    object
 1   query_text  500 non-null    object
 2   image       500 non-null    object
dtypes: object(3)
memory usage: 15.6+ KB



test_images


,image
0,3356748019_2251399314.jpg
1,2887171449_f54a2b9f39.jpg
2,3089107423_81a24eaf18.jpg
3,1429546659_44cb09cbe2.jpg
4,1177994172_10d143cb8d.jpg
5,424307754_1e2f44d265.jpg
6,3044359043_627488ddf0.jpg
7,3396275223_ee080df8b5.jpg
8,2977379863_2e8d7a104e.jpg
9,634891010_9fa189effb.jpg


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   100 non-null    object
dtypes: object(1)
memory usage: 932.0+ bytes





Добавим в файл обобщенную экспертную оценку. Для этого создадим функцию, которая будет возвращать итоговую оценку или ошибку, если выставление итоговой оценки невозможно.

In [5]:
def result_exp(str_in):
    res=[0,0,0,0]
    for i in range(0,3):
        res[int(str_in[i+2]-1)]+=1
    if max(res) == 1:
        return None
    else:
        try:
            return res.index(max(res))+1
        except:
            print('ERROR')
            return None

Добавим колонку с результатом обобщения в датасет с экспертными оценками.

In [6]:
ExpertAnnotations[5] = ExpertAnnotations.apply(result_exp, axis=1)
ExpertAnnotations = ExpertAnnotations.dropna()

In [7]:
ExpertAnnotations.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5696 entries, 0 to 5821
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       5696 non-null   object 
 1   1       5696 non-null   object 
 2   2       5696 non-null   int64  
 3   3       5696 non-null   int64  
 4   4       5696 non-null   int64  
 5   5       5696 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 311.5+ KB


In [8]:
train_dataset = train_dataset.iloc[ExpertAnnotations.index]

## 2. Проверка данных

В некоторых странах, где работает ваша компания, действуют ограничения по обработке изображений: поисковым сервисам и сервисам, предоставляющим возможность поиска, запрещено без разрешения родителей или законных представителей предоставлять любую информацию, в том числе, но не исключительно тексты, изображения, видео и аудио, содержащие описание, изображение или запись голоса детей. Ребёнком считается любой человек, не достигший 16 лет.

В вашем сервисе строго следуют законам стран, в которых работают. Поэтому при попытке посмотреть изображения, запрещённые законодательством, вместо картинок показывается дисклеймер:

> This image is unavailable in your country in compliance with local laws
>

Однако у вас в PoC нет возможности воспользоваться данным функционалом. Поэтому все изображения, которые нарушают данный закон, нужно удалить из обучающей выборки.

По условию помимо стоп-слов из датасета нужно исключить информацию о детях и подростках, чтобы избежать нарушения законодательства. Создадим для этого словарь с основными словами, связанными с подростками/школьниками/детьми.

In [9]:
ban_dict = ['child', 
             'children',
             'baby', 
             'babies',
             'boy',
             'boys',
             'girl',
             'girls',
             'kid',
             'kids',
             'infant',
             'infants',
             'toddler',
             'toddlers',
             'teenager',
             'teenagers',
             'schoolboy',
             'schoolboys',
             'youth',
             'newborn',
             'newborns']

Загрузим стоп-слова и лемматизируем текст описания

In [10]:
nlp = spacy.load("en_core_web_sm")

def lemma_clear(text):
    lemm = nlp(text)
    lemm = " ".join([token.lemma_ for token in lemm])


    return " ".join(lemm.split())

Добавим в датасет столбец с очищенным и лемматизированным текстом.

In [11]:
corpus = train_dataset['query_text'].apply(lemma_clear)

Теперь исключим из датасета слова, связанные с несовершеннолетними. Флаг того, что такие слова есть в описании фотографии, добавим в отдельный столбец датасета.

In [12]:
def is_ban_word(text):
    for s in ban_dict: 
        if text.find(s) > -1:
            return False
    return True

In [96]:
print(is_ban_word('youth'))

False


In [13]:
train_dataset['lem_query_text'] = corpus
train_dataset['is_in_law'] = corpus.apply(is_ban_word)

In [14]:
def spl(text):
    return text[:text.find('#')]

In [15]:
#формируем список фото, которые содержат в соём описании упоминание несовершеннолетних
forb_images = list(train_dataset[train_dataset['is_in_law']==False]['query_id'].apply(spl).unique())

In [16]:
len(forb_images)

296

In [98]:
forb_images

['2549968784_39bfbe44f9.jpg',
 '2718495608_d8533e3ac5.jpg',
 '3545652636_0746537307.jpg',
 '1714316707_8bbaa2a2ba.jpg',
 '2140182410_8e2a06fbda.jpg',
 '2479652566_8f9fac8af5.jpg',
 '1287475186_2dee85f1a5.jpg',
 '2358554995_54ed3baa83.jpg',
 '2544426580_317b1f1f73.jpg',
 '191003285_edd8d0cf58.jpg',
 '2370481277_a3085614c9.jpg',
 '2577972703_a22c5f2a87.jpg',
 '3388330419_85d72f7cda.jpg',
 '2274992140_bb9e868bb8.jpg',
 '2445283938_ff477c7952.jpg',
 '2510020918_b2ca0fb2aa.jpg',
 '3325497914_f9014d615b.jpg',
 '3549583146_3e8bb2f7e9.jpg',
 '1131932671_c8d17751b3.jpg',
 '3245460937_2710a82709.jpg',
 '3516825206_5750824874.jpg',
 '3239021459_a6b71bb400.jpg',
 '3459156091_c1879ebe28.jpg',
 '1237985362_dbafc59280.jpg',
 '2594042571_2e4666507e.jpg',
 '533713007_bf9f3e25b4.jpg',
 '543007912_23fc735b99.jpg',
 '2295750198_6d152d7ceb.jpg',
 '3185409663_95f6b958d8.jpg',
 '2228022180_9597b2a458.jpg',
 '3243588540_b418ac7eda.jpg',
 '3280052365_c4644bf0a5.jpg',
 '3470951932_27ed74eb0b.jpg',
 '2501595799_

Таких изображений оказалось 296

In [17]:
def is_not_forb(text):
    if text in forb_images:
        return False
    else:
        return True
    
train_dataset_clear = train_dataset[train_dataset['is_in_law']]
train_dataset_clear = train_dataset_clear[train_dataset_clear['image'].apply(is_not_forb)]

In [94]:
train_dataset[train_dataset['is_in_law']]

,image,query_id,query_text,lem_query_text,is_in_law
22,1056338697_4f7d7ce270.jpg,3181701312_70a379ab6e.jpg#2,A man sleeps under a blanket on a city street .,a man sleep under a blanket on a city street .,True
23,3187395715_f2940c2b72.jpg,3181701312_70a379ab6e.jpg#2,A man sleeps under a blanket on a city street .,a man sleep under a blanket on a city street .,True
24,463978865_c87c6ca84c.jpg,3181701312_70a379ab6e.jpg#2,A man sleeps under a blanket on a city street .,a man sleep under a blanket on a city street .,True
25,488590040_35a3e96c89.jpg,3181701312_70a379ab6e.jpg#2,A man sleeps under a blanket on a city street .,a man sleep under a blanket on a city street .,True
26,534875358_6ea30d3091.jpg,3181701312_70a379ab6e.jpg#2,A man sleeps under a blanket on a city street .,a man sleep under a blanket on a city street .,True
...,...,...,...,...,...
5816,799486353_f665d7b0f0.jpg,2170222061_e8bce4a32d.jpg#2,"A small animal leaps behind a larger animal , ...","a small animal leap behind a large animal , th...",True
5818,799486353_f665d7b0f0.jpg,2196107384_361d73a170.jpg#2,a old man walks down the uncrowded road .,a old man walk down the uncrowded road .,True
5819,909808296_23c427022d.jpg,2112921744_92bf706805.jpg#2,A dog stands on the side of a grassy cliff .,a dog stand on the side of a grassy cliff .,True
5820,929679367_ff8c7df2ee.jpg,3651971126_309e6a5e22.jpg#2,A blurry photo of two dogs .,a blurry photo of two dog .,True


Добавим оценки экспертов в датасет, предварительно пересчитав их, так как оценки экспертов имеют 4 уровня.

In [18]:
display(ExpertAnnotations.head())
display(CrowdAnnotations.head())

,0,1,2,3,4,5
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,1,1,1,1.0
1,1056338697_4f7d7ce270.jpg,2718495608_d8533e3ac5.jpg#2,1,1,2,1.0
2,1056338697_4f7d7ce270.jpg,3181701312_70a379ab6e.jpg#2,1,1,2,1.0
3,1056338697_4f7d7ce270.jpg,3207358897_bfa61fa3c6.jpg#2,1,2,2,2.0
4,1056338697_4f7d7ce270.jpg,3286822339_5535af6b93.jpg#2,1,1,2,1.0


,0,1,2,3,4
0,1056338697_4f7d7ce270.jpg,1056338697_4f7d7ce270.jpg#2,1.0,3,0
1,1056338697_4f7d7ce270.jpg,114051287_dd85625a04.jpg#2,0.0,0,3
2,1056338697_4f7d7ce270.jpg,1427391496_ea512cbe7f.jpg#2,0.0,0,3
3,1056338697_4f7d7ce270.jpg,2073964624_52da3a0fc4.jpg#2,0.0,0,3
4,1056338697_4f7d7ce270.jpg,2083434441_a93bc6306b.jpg#2,0.0,0,3


In [19]:
ExpertAnnotations[6] = (ExpertAnnotations[5]-1)/3

In [24]:
train_data = ExpertAnnotations.merge(CrowdAnnotations, left_on=[0, 1], right_on=[0, 1])[[0, 1, 6, '2_y']]
train_data['final_score'] = train_data[6]*0.6 + train_data['2_y']*0.4
train_data

,0,1,6,2_y,final_score
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,0.000000,0.000000,0.000000
1,1056338697_4f7d7ce270.jpg,2718495608_d8533e3ac5.jpg#2,0.000000,0.000000,0.000000
2,1056338697_4f7d7ce270.jpg,434792818_56375e203f.jpg#2,0.000000,0.000000,0.000000
3,1084040636_97d9633581.jpg,256085101_2c2617c5d0.jpg#2,0.666667,0.333333,0.533333
4,1084040636_97d9633581.jpg,3396157719_6807d52a81.jpg#2,0.333333,0.000000,0.200000
...,...,...,...,...,...
2253,979383193_0a542a059d.jpg,3244747165_17028936e0.jpg#2,0.333333,0.000000,0.200000
2254,979383193_0a542a059d.jpg,3482062809_3b694322c4.jpg#2,0.333333,0.000000,0.200000
2255,997722733_0cb5439472.jpg,2985679744_75a7102aab.jpg#2,0.000000,0.000000,0.000000
2256,997722733_0cb5439472.jpg,3150742439_b8a352e1e0.jpg#2,0.000000,0.000000,0.000000


In [28]:
train_data.columns = ['image', 'query_id','expert_score', 'croud_score', 'final_score']

In [97]:
train_dataset_clear

,image,query_id,query_text,lem_query_text,is_in_law
0,463978865_c87c6ca84c.jpg,3181701312_70a379ab6e.jpg#2,A man sleeps under a blanket on a city street .,a man sleep under a blanket on a city street .,True
1,370713359_7560808550.jpg,3207358897_bfa61fa3c6.jpg#2,A woman plays with long red ribbons in an empt...,a woman play with long red ribbon in an empty ...,True
2,2101457132_69c950bc45.jpg,3286822339_5535af6b93.jpg#2,Chinese market street in the winter time .,chinese market street in the winter time .,True
3,1056338697_4f7d7ce270.jpg,434792818_56375e203f.jpg#2,A man and woman look back at the camera while ...,a man and woman look back at the camera while ...,True
4,132489044_3be606baf7.jpg,434792818_56375e203f.jpg#2,A man and woman look back at the camera while ...,a man and woman look back at the camera while ...,True
...,...,...,...,...,...
1420,415793623_6c1225ae27.jpg,2554081584_233bdf289a.jpg#2,a man wearing a yellow shirt and white hardhat...,a man wear a yellow shirt and white hardhat st...,True
1421,476759700_8911f087f8.jpg,1082379191_ec1e53f996.jpg#2,A man and a woman sitting on a dock .,a man and a woman sit on a dock .,True
1422,561417861_8e25d0c0e8.jpg,410453140_5401bf659a.jpg#2,A man stands at the entrance of Yerba Buena Ce...,a man stand at the entrance of Yerba Buena Cen...,True
1423,561940436_64d6fc125d.jpg,3157847991_463e006a28.jpg#2,Two men in black jackets behind a white handra...,two man in black jacket behind a white handrail .,True


In [29]:
train_data

,image,query_id,expert_score,croud_score,final_score
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,0.000000,0.000000,0.000000
1,1056338697_4f7d7ce270.jpg,2718495608_d8533e3ac5.jpg#2,0.000000,0.000000,0.000000
2,1056338697_4f7d7ce270.jpg,434792818_56375e203f.jpg#2,0.000000,0.000000,0.000000
3,1084040636_97d9633581.jpg,256085101_2c2617c5d0.jpg#2,0.666667,0.333333,0.533333
4,1084040636_97d9633581.jpg,3396157719_6807d52a81.jpg#2,0.333333,0.000000,0.200000
...,...,...,...,...,...
2253,979383193_0a542a059d.jpg,3244747165_17028936e0.jpg#2,0.333333,0.000000,0.200000
2254,979383193_0a542a059d.jpg,3482062809_3b694322c4.jpg#2,0.333333,0.000000,0.200000
2255,997722733_0cb5439472.jpg,2985679744_75a7102aab.jpg#2,0.000000,0.000000,0.000000
2256,997722733_0cb5439472.jpg,3150742439_b8a352e1e0.jpg#2,0.000000,0.000000,0.000000


In [30]:
train_dataset_clear = train_dataset_clear.merge(train_data, left_on=['image', 'query_id'], right_on=['image', 'query_id'])[train_dataset_clear.columns]
train_data = train_dataset_clear.merge(train_data, left_on=['image', 'query_id'], right_on=['image', 'query_id'])[train_data.columns]

In [106]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1425 entries, 0 to 1424
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   image         1425 non-null   object 
 1   query_id      1425 non-null   object 
 2   expert_score  1425 non-null   float64
 3   croud_score   1425 non-null   float64
 4   final_score   1425 non-null   float64
dtypes: float64(3), object(2)
memory usage: 55.8+ KB


In [104]:
train_data['image'] = train_data['image'].astype('str')

In [114]:
flag = 0
for word in train_data['image']:
    if word in forb_images:
        print(word)
    else:
        flag=flag+1

In [115]:
flag

1425

## 3. Векторизация изображений

Перейдём к векторизации изображений.

Самый примитивный способ — прочесть изображение и превратить полученную матрицу в вектор. Такой способ нам не подходит: длина векторов может быть сильно разной, так как размеры изображений разные. Поэтому стоит обратиться к свёрточным сетям: они позволяют "выделить" главные компоненты изображений. Как это сделать? Нужно выбрать какую-либо архитектуру, например ResNet-18, посмотреть на слои и исключить полносвязные слои, которые отвечают за конечное предсказание. При этом можно загрузить модель данной архитектуры, предварительно натренированную на датасете ImageNet.

In [43]:
resnet = models.resnet50(pretrained=True)

for param in resnet.parameters():
    param.requires_grad_(False)
    
print(list(resnet.children())) 

modules = list(resnet.children())[:-1]
resnet = nn.Sequential(*modules) 

resnet.eval() 

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False), BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False), Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=Tr

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [44]:
norm = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

preprocess = transforms.Compose([
    transforms.Resize(150),
    transforms.CenterCrop(140),
    transforms.ToTensor(),
    norm,
]) 

In [45]:
def vect_gen_test(string):
    img = Image.open(path+string[0]).convert('RGB')
    image_tensor = preprocess(img)
    output_tensor = resnet(image_tensor.unsqueeze(0)).flatten()
    output_tensor.size()
    return output_tensor.numpy()

def vect_gen_train(string):
    img = Image.open(path+string['image']).convert('RGB')
    image_tensor = preprocess(img)
    output_tensor = resnet(image_tensor.unsqueeze(0)).flatten()
    output_tensor.size()
    return output_tensor.numpy()

In [40]:
path="to_upload/train_images/"
train_img_vec_arr = np.array(train_data.apply(vect_gen_test, axis=1))

torch.Size([1000])

In [41]:
def vect_gen_test(string):
    img = Image.open(path+string[0]).convert('RGB')
    image_tensor = preprocess(img)
    output_tensor = resnet(image_tensor.unsqueeze(0)).flatten()
    output_tensor.size()
    return output_tensor.numpy()

def vect_gen_train(string):
    img = Image.open(path+string['image']).convert('RGB')
    image_tensor = preprocess(img)
    output_tensor = resnet(image_tensor.unsqueeze(0)).flatten()
    output_tensor.size()
    return output_tensor.numpy()

In [46]:
path="to_upload/train_images/"
train_img_vec_arr = np.array(train_data.apply(vect_gen_test, axis=1))

## 4. Векторизация текстов

Следующий этап — векторизация текстов. Вы можете поэкспериментировать с несколькими способами векторизации текстов:

- tf-idf
- word2vec
- \*трансформеры (например Bert)

\* — если вы изучали трансформеры в спринте Машинное обучение для текстов.


Векторизируем тексты с помощью tf-idf

In [47]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Эльдорадо\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
corpus = train_dataset_clear['lem_query_text']
count_tf_idf = TfidfVectorizer(stop_words=list(stopwords)) 
tf_idf_train = count_tf_idf.fit_transform(corpus) 
print("Размер матрицы:", tf_idf_train.shape)

Размер матрицы: (1425, 740)


In [49]:
corpus = test_queries['query_text'].apply(lemma_clear)
test_queries['lem_query_text'] = corpus
corpus = test_queries['lem_query_text']
tf_idf_test = count_tf_idf.transform(corpus) 
print("Размер матрицы:", tf_idf_test.shape)

Размер матрицы: (500, 740)


## 5. Объединение векторов

Подготовьте данные для обучения: объедините векторы изображений и векторы текстов с целевой переменной.

In [50]:
train_united = np.hstack([np.vstack(train_img_vec_arr), tf_idf_train.toarray()])

## 6. Обучение модели предсказания соответствия

Для обучения разделите датасет на тренировочную и тестовую выборки. Простое случайное разбиение не подходит: нужно исключить попадание изображения и в обучающую, и в тестовую выборки.
Для того чтобы учесть изображения при разбиении, можно воспользоваться классом [GroupShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupShuffleSplit.html) из библиотеки sklearn.model_selection.

Код ниже разбивает датасет на тренировочную и тестовую выборки в пропорции 7:3 так, что строки с одинаковым значением 'group_column' будут содержаться либо в тестовом, либо в тренировочном датасете.

```
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, train_size=.7, random_state=42)
train_indices, test_indices = next(gss.split(X=df.drop(columns=['target']), y=df['target'], groups=df['group_column']))
train_df, test_df = df.loc[train_indices], df.loc[test_indices]

```

Какую модель использовать — выберите самостоятельно. Также вам предстоит выбрать метрику качества либо реализовать свою.

In [53]:
gss = GroupShuffleSplit(n_splits=1, train_size=.7, random_state=42)
train_indexes, test_indexes = next(gss.split(X=train_united, y=train_data['final_score'], groups=train_dataset_clear['image']))

X_train = train_united[train_indexes]
X_test = train_united[test_indexes]
y_train = train_data['final_score'].loc[train_indexes]
y_test = train_data['final_score'].loc[test_indexes]

In [54]:
model_lr = LinearRegression(positive=True)
model_lr.fit(X_train, y_train)

LinearRegression(positive=True)

In [56]:
scaler = MinMaxScaler()

print("RMSE кросс-валидации:", 
      mean_squared_error(y_test, scaler.fit_transform(model_lr.predict(X_test).reshape(-1, 1)), squared=False))

RMSE кросс-валидации: 0.4193912193421502


In [57]:
X_train = torch.FloatTensor(np.array(X_train))
X_test = torch.FloatTensor(np.array(X_test))
y_train = torch.FloatTensor(np.array(y_train))
y_test = torch.FloatTensor(np.array(y_test))

In [58]:
X_train.shape

torch.Size([958, 2788])

In [59]:
class Net(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, n_hidden_neurons_3, n_hidden_neurons_4, n_out_neurons):
            super(Net, self).__init__()
            
            self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
            self.act1 = nn.ReLU()
            self.fc2 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2)
            self.act2 = nn.ReLU()
            self.fc3 = nn.Linear(n_hidden_neurons_2, n_hidden_neurons_3)
            self.act3 = nn.ReLU()
            self.fc4 = nn.Linear(n_hidden_neurons_3, n_hidden_neurons_4)
            self.act4 = nn.ReLU()
            self.fc5 = nn.Linear(n_hidden_neurons_4, n_out_neurons)
            self.act5 = nn.Sigmoid()
            
            nn.init.kaiming_uniform_(self.fc1.weight, mode='fan_in', nonlinearity='relu')
            nn.init.kaiming_uniform_(self.fc2.weight, mode='fan_in', nonlinearity='relu')
            nn.init.kaiming_uniform_(self.fc3.weight, mode='fan_in', nonlinearity='relu')
            nn.init.kaiming_uniform_(self.fc4.weight, mode='fan_in', nonlinearity='relu')
            nn.init.kaiming_uniform_(self.fc5.weight, mode='fan_in', nonlinearity='sigmoid')
            
    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.fc3(x)
        x = self.act3(x)
        x = self.fc4(x)
        x = self.act4(x)
        x = self.fc5(x)
        x = self.act5(x)
        
        return x

In [60]:
n_in_neurons = 2788
n_hidden_neurons_1 = 1200
n_hidden_neurons_2 = 600
n_hidden_neurons_3 = 200
n_hidden_neurons_4 = 10
n_out_neurons = 1

net = Net(n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, n_hidden_neurons_3, n_hidden_neurons_4, n_out_neurons)

optimizer2 = torch.optim.SGD(net.parameters(), lr=1e-3, momentum=0.8)

optimizers = [optimizer2]

loss = nn.MSELoss()

In [61]:
num_epochs = 900
batch_size = 150
num_batches = ceil(len(X_train)/batch_size)

for optimizer in optimizers:
    optimizer_rmse = []
    print(f'Оптимизатор {optimizer}:')

    for epoch in range(num_epochs):
        net.train()
        order = np.random.permutation(len(X_train))
        
        for batch_idx in range(num_batches):
            
            start_index = batch_idx * batch_size
            optimizer.zero_grad()

            batch_indexes = order[start_index : start_index + batch_size]
            X_batch = X_train[batch_indexes]
            y_batch = y_train[batch_indexes]

            preds = net.forward(X_batch.float()).flatten()

            loss_value = loss(preds, y_batch)

            loss_value.backward()

            optimizer.step()

        if epoch % 10 == 0 or epoch == num_epochs - 1:
            with torch.no_grad():
                net.eval()
                test_preds = net.forward(X_test).flatten()
                #print(test_preds.numpy())
                ans = round(float(torch.sqrt(loss(test_preds, y_test))), 2)
                print(f'Метрика RMSE на {epoch} итерации =', ans)
                optimizer_rmse.append(ans)
    print()
    print(f'Лучшая метрика RMSE = {min(optimizer_rmse)} обнаружена на {optimizer_rmse.index(min(optimizer_rmse))} эпохе')
    print(f'Средняя метрика RMSE = {round(np.mean(optimizer_rmse), 2)}')

Оптимизатор SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.8
    nesterov: False
    weight_decay: 0
):
Метрика RMSE на 0 итерации = 0.44
Метрика RMSE на 10 итерации = 0.33
Метрика RMSE на 20 итерации = 0.33
Метрика RMSE на 30 итерации = 0.33
Метрика RMSE на 40 итерации = 0.33
Метрика RMSE на 50 итерации = 0.32
Метрика RMSE на 60 итерации = 0.32
Метрика RMSE на 70 итерации = 0.32
Метрика RMSE на 80 итерации = 0.32
Метрика RMSE на 90 итерации = 0.32
Метрика RMSE на 100 итерации = 0.32
Метрика RMSE на 110 итерации = 0.32
Метрика RMSE на 120 итерации = 0.32
Метрика RMSE на 130 итерации = 0.32
Метрика RMSE на 140 итерации = 0.32
Метрика RMSE на 150 итерации = 0.32
Метрика RMSE на 160 итерации = 0.32
Метрика RMSE на 170 итерации = 0.32
Метрика RMSE на 180 итерации = 0.32
Метрика RMSE на 190 итерации = 0.33
Метрика RMSE на 200 итерации = 0.33
Метрика RMSE на 210 итерации = 0.33
Метрика RMSE на 220 итераци

## 7. Тестирование модели

Настало время протестировать модель. Для этого получите эмбеддинги для всех тестовых изображений из папки `test_images`, выберите случайные 10 запросов из файла `test_queries.csv` и для каждого запроса выведите наиболее релевантное изображение. Сравните визуально качество поиска.

In [62]:
path="to_upload/test_images/"
test_images_vec = np.vstack(np.array(test_images.apply(vect_gen_train, axis=1)))

In [90]:
def ten_texts():
    texts = []
    for i in range(0,10):
        texts.append(test_queries.iloc[rd.randint(0, 499)]['query_text'])
    return texts

In [116]:
is_ban_word('myword')

True

In [119]:
def conv_text(text):
    corpus_one = lemma_clear(text)
    corpus = []
    if not is_ban_word(corpus_one): 
        return None
    else:
        for i in range(0,100):
            corpus.append(corpus_one)
        tf_idf_one = count_tf_idf.transform(corpus)
        return(np.hstack([np.vstack(test_images_vec), tf_idf_one.toarray()]))

In [148]:
tt_arr=ten_texts()

In [156]:
flag=0
for item in tt_arr:  #test_queries['query_text']: #ten_texts():
    try:
        mass = conv_text(item)
        res = net.forward(torch.FloatTensor(mass)).flatten().detach().numpy()
        print(item)
        print(len(res))
        print(res.argmax())
        print(mass[res.argmax()])
        
        break
    except:
        flag=flag+1
        

A small white dog is carrying something blue in its mouth and a bigger brown and black dog is carrying something orange in its mouth .
100
50
[0.19113514 0.90533769 0.43438357 ... 0.         0.         0.        ]


In [154]:
print(tt_arr[0])
print(conv_text(tt_arr[0]))

A small white dog is carrying something blue in its mouth and a bigger brown and black dog is carrying something orange in its mouth .
[[0.1357713  1.19895554 0.1268108  ... 0.         0.         0.        ]
 [0.68877518 2.05044127 3.07429576 ... 0.         0.         0.        ]
 [0.05477677 0.12885231 0.32981056 ... 0.         0.         0.        ]
 ...
 [0.11907032 1.92410982 0.61920911 ... 0.         0.         0.        ]
 [0.15232837 0.88904953 1.323524   ... 0.         0.         0.        ]
 [0.08256899 0.70489466 0.24657874 ... 0.         0.         0.        ]]


In [174]:
for item in test_queries['query_text']:
    print(item)
    try:
        mass = conv_text(item)
        res = net.forward(torch.FloatTensor(mass)).flatten().detach().numpy()
        #print(res)
        print(res.argmax())
        #print(mass.shape)
    except:
        print('В запросе присутствует запретное слово!')

Two blonde boys , one in a camouflage shirt and the other in blue , are having a water fight .
В запросе присутствует запретное слово!
Two boys are squirting water guns at each other .
В запросе присутствует запретное слово!
Two boys spraying each other with water
В запросе присутствует запретное слово!
Two children wearing jeans squirt water at each other .
В запросе присутствует запретное слово!
Two young boys are squirting water at each other .
В запросе присутствует запретное слово!
A baby girl playing at a park .
В запросе присутствует запретное слово!
A closeup of a child on a playground with adult supervision .
В запросе присутствует запретное слово!
A young boy poses for a picture in front of a playground .
В запросе присутствует запретное слово!
A young girl is smiling in front of the camera at a park .
В запросе присутствует запретное слово!
There is a little blond hair girl with a green sweatshirt and a red shirt playing on a playground .
В запросе присутствует запретное сло

50
An old woman with a white dog
50
A white bulldog sitting on a tree stump gives paw to an older woman wearing a black jacket .
50
A woman and a dog sit on a tree stump .
50
There is an older white haired lady and a bulldog sitting on a tree stump .
50
A man , seen from the back , with his head turned to the side wearing a backpack
50
A man wearing a backpack looking to the side .
50
An adult male wearing a backpack walks and looks to the left .
50
The man in the backpack looks to his left .
50
the man is wearing a backpack and sunglasses while walking through an outdoor market .
50
A boy flying a red and white kite .
В запросе присутствует запретное слово!
A boy is flying a kite and smiling at the camera .
В запросе присутствует запретное слово!
A child flying a kite .
В запросе присутствует запретное слово!
A little boy is flying a kite high up in the air .
В запросе присутствует запретное слово!
Boy flying a distant kite .
В запросе присутствует запретное слово!
a tan dog jumping t

50
A man wearing glasses and black clothing is sitting while a man in a vest drinks in the background .
50
A man with glasses wearing all black sits on grass surrounded by other people .
50
A salt-and-pepper-haired man with beard and glasses wearing black sits on the grass .
50
Closeup of a man at an event with formal attire .
64
The man is dressed in black and smiling while another man is drinking and an elderly woman is standing near a fence .
50
A man in a black vest walks on a rail .
50
A man is standing with his arms spread wide and an open vest exposing his chest .
50
A man wearing a vest and pants walks across stage at a concert .
64
A man who is dressed like a wrestler walks while people take his picture .
50
A shirtless man wearing a vest walks on a stage with his arms up .
50
A playful puppy bites the shirt of a young boy .
В запросе присутствует запретное слово!
Two boys and a white dog stand in a park .
В запросе присутствует запретное слово!
Two children play with a white 

50
A man sits on the gravel by an ocean .
50
A young man sits on a cliff , watching the ocean .
50
Seated man looks out at the ocean .
50
A bicyclist is jumping on ramp covered with graffiti .
50
A man is jumping his bicycle off of a ramp .
50
A skateboarder jumping on a graffiti covered ramp .
50
A young boy jumps his bicycle on a tagged bike ramp .
В запросе присутствует запретное слово!
Young man on a trick bicycle jumps at the top of a graffiti covered ramp .
50
A dog fetching a stick .
50
a little white and tan dog running through the grass with a stick in his mouth .
50
A little white dog runs on freshly mown grass , holding a small stick .
50
The small dog returns with his stick proudly .
50
The small fluffy white dog runs through the green grass with a tiny stick in its mouth .
50
A group of girls wearing costumes stand in a park .
В запросе присутствует запретное слово!
A group of girls who are gothically dressed standing together
В запросе присутствует запретное слово!
A grou

50
A girl is wearing a blue wig under a brown hooded jacket .
В запросе присутствует запретное слово!
A man wears an interesting blue wig in public
50
A person with blue hair , a baseball cap , and a hood on stands outdoors in a crowd .
50
Man with brown coat and sunglasses wears a blue wig and ball cap .
50
One person in a crowd of people , wearing a blue wig
50
A brown dog and a black dog run into each other in the grass .
50
A brown dog is grabbing the collar of a black dog .
50
A dark black dog is playing with a light brown dog in a backyard .
50
Two dogs are wrestling in a field .
50
Two dogs are wrestling in the park .
50
A little kid splashes around in the kiddie pool while a lady watches .
В запросе присутствует запретное слово!
a woman dumping water on a small child who is in a pool
В запросе присутствует запретное слово!
A woman is throwing a bucket of water over a baby in the paddling pool .
В запросе присутствует запретное слово!
A woman is throwing water on a child in a pl

Модель работает и проверяет вводимый запрос на наличие стоп-слов, однако в качестве наиболее соответствующей выдаёт только 2 картинки.

## 8. Выводы

- Jupyter Notebook открыт
- Весь код выполняется без ошибок
- Ячейки с кодом расположены в порядке исполнения
- Исследовательский анализ данных выполнен
- Проверены экспертные оценки и краудсорсинговые оценки
- Из датасета исключены те объекты, которые выходят за рамки юридических ограничений
- Изображения векторизованы
- Текстовые запросы векторизованы
- Данные корректно разбиты на тренировочную и тестовую выборки
- Предложена метрика качества работы модели
- Предложена модель схожести изображений и текстового запроса
- Модель обучена
- По итогам обучения модели сделаны выводы
- Проведено тестирование работы модели
- По итогам тестирования визуально сравнили качество поиска